In [5]:
# this parses and stores the output as a dataframe. The correct prompt is:
# Generate a multiple-choice question for the following passage. First, write 'Question:' and write the question. Then write the answer options. Then write 'Correct Answer:' and give the correct answer. Finally, write explanations for the correct answer and each of the distractors on a new line starting with 'Explanation for A'.

In [1]:
import pandas as pd
import os
import re
import openai
import os
from getpass import getpass
import json
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

Create a .env file in `/src` with the follow line:
`API_TOKEN={your api key}`

I added `**/.env` to `.gitignore` already. In the future, you'll have to double check that git is ignoring the secrets file.

You may need to `pip install python-dotenv`

In [2]:
%load_ext dotenv
%dotenv
import os
load_dotenv(Path(".env"))
openai_api_key = os.getenv('API_TOKEN').strip('{}')

In [3]:
DATA = '../data/'
subsections = pd.read_csv(DATA + 'subsections.csv')


In [4]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
model = OpenAI(temperature=0.9, openai_api_key = openai_api_key)

## Short Answer Questions

In [52]:
# The types of questions we are interested in
kinds = {'recall': 'requiring the reader to recall information from the text',
         'summarization': 'requiring the reader to summarize information from the text',
         'inference': 'requiring the reader to make inferences from the text'}

In [58]:
# Create a response schema to organize the output from the model
response_schemas = [
    ResponseSchema(name="question", description="a question based on the passage. The answer should be one sentence long."),
    ResponseSchema(name="correct_answer", description="a correct short answer to the question. The answer should be one sentence long."),
    ResponseSchema(name="incorrect_answer", description="an incorrect short answer to the question. The answer should be one sentence long.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [59]:
# Create a prompt template, the input to feed to the model
prompt = PromptTemplate(
    template="generate a question based on the passage {kind}. \n{format_instructions}\n{passage}",
    input_variables=["kind", "passage"],
    partial_variables={"format_instructions": format_instructions}
)

In [76]:
'''
This is a function which takes a passage and a desired type of question, as well as the chapter, section, and subsection numbers
It returns a dataframe with one row
'''

def generate_sa_question(example, kind):
    # Input to the model
    _input = prompt.format_prompt(passage=example['clean_text'], kind=kind)
    output = model(_input.to_string())
    
    # Clean out the output string and convert it to a dataframe
    output_string = output.replace('`','').replace('json','').strip()
    # Try to decode the json
    try:
        output_dict = json.loads(output_string)
    # If it doesn't work, return an empty row
    except:
        output_dict = {'question':'', 'correct_answer':'', 'incorrect_answer':''}
        print('Error Parsing Output')

    output_dict['type'] = kind
    return output_dict

In [77]:
import datasets

ds = datasets.Dataset.from_pandas(subsections)
ds_dict = {}

for k in kinds:
    ds_dict[k] = ds.map(lambda x: generate_sa_question(x, k))

ds_dict

Map:   3%|▎         | 16/523 [01:48<1:06:18,  7.85s/ examples]

Error Parsing Output


Map:   7%|▋         | 36/523 [04:17<1:03:43,  7.85s/ examples]

Error Parsing Output


Map:  11%|█         | 57/523 [06:41<55:24,  7.13s/ examples]  

...

Map:  96%|█████████▌| 503/523 [1:15:28<03:25, 10.29s/ examples]

Error Parsing Output


Map:  98%|█████████▊| 512/523 [1:16:50<01:39,  9.08s/ examples]

Error Parsing Output


{'recall': Dataset({
     features: ['module', 'chapter', 'section', 'subsection', 'heading', 'raw_text', 'clean_text', 'slug', 'question', 'correct_answer', 'incorrect_answer', 'type'],
     num_rows: 523
 }),
 'summarization': Dataset({
     features: ['module', 'chapter', 'section', 'subsection', 'heading', 'raw_text', 'clean_text', 'slug', 'question', 'correct_answer', 'incorrect_answer', 'type'],
     num_rows: 523
 }),
 'inference': Dataset({
     features: ['module', 'chapter', 'section', 'subsection', 'heading', 'raw_text', 'clean_text', 'slug', 'question', 'correct_answer', 'incorrect_answer', 'type'],
     num_rows: 523
 })}

In [92]:
df_list = [v.to_pandas() for v in ds_dict.values()]
big_df = pd.concat(df_list)
big_df.reset_index().to_csv('../data/aqag-chatcpt.csv')

In [91]:
big_df1 = big_df
big_df1[big_df1['index'] == 3]

,index,module,chapter,section,subsection,heading,raw_text,clean_text,slug,question,correct_answer,incorrect_answer,type
3,3,1,1,1,1,Introduction to FRED,Data is very important in economics because it...,Data is very important in economics because it...,introduction-to-fred,What is the name of the database where most of...,The St. Louis Federal Reserve Bank's FRED data...,The US Census Bureau database.,recall
526,3,1,1,1,1,Introduction to FRED,Data is very important in economics because it...,Data is very important in economics because it...,introduction-to-fred,What website does the course primarily use for...,The course primarily uses the St. Louis Federa...,The course primarily uses the St. Louis Federa...,summarization
1049,3,1,1,1,1,Introduction to FRED,Data is very important in economics because it...,Data is very important in economics because it...,introduction-to-fred,Which government agencies are sources of econo...,A variety of government agencies,FRED,inference


In [62]:
'''
This cell generates short answer questions and correct and incorrect answer for each row of the subsections dataframe
'''

df = pd.DataFrame({'clean_text':[], 'question':[], 'correct_answer':[], 'incorrect_answer':[], 'type':[], 'chapter':[], 'section':[], 'subsection':[]})
# Count how many rows chatGPT failed at
problem_rows = 0

for row in subsections.iterrows():
    print(row[0], end='\r')
    passage = row[1]['clean_text']
    for kind in kinds:
        try: 
            line = generate_sa_question(passage, kind, row[1]['chapter'], row[1]['section'], row[1]['subsection'])
            df = pd.concat([df, line], axis=0)
        except:
            problem_rows += 1
            continue

print(f'Trouble parsing {problem_rows} subsections')
        
        

Trouble parsing 1569 subsections


In [ ]:
df

In [ ]:
df.reset_index(drop=True).to_csv('../results/aqg.csv')

## Multiple Choice Questions

In [ ]:
template = """
I want you to generate a multiple choice question for a passage with one correct answer and explanations for all answers using this as an example:

Question:

A:
B:
C:
D:

Correct Answer:

Explanation for A:
Explanation for B:
Explanation for C:
Explanation for D:

Here is the passage I want you to use to generate the question: {passage}
"""

prompt = PromptTemplate(
    input_variables=["passage"],
    template = template)

questions_dict = {'id':[], 'chapter':[], 'section':[], 'subsection':[], 'item':[]}
counter = 0
for row in subsections.iterrows():
    print(counter, end='\r')
    questions_dict['id'].append(row[0])
    questions_dict['chapter'].append(row[1]['chapter'])
    questions_dict['section'].append(row[1]['section'])
    questions_dict['subsection'].append(row[1]['subsection'])
    questions_dict['item'].append(llm(prompt.format(passage=row[1]['clean_text'])))
    counter +=1


In [ ]:
questions_df = pd.DataFrame.from_dict(questions_dict)
questions_df.to_csv('../data/gpt_questions.csv')

In [6]:
def generate_question(summary):
    prompt = """Generate a multiple-choice question, give the correct answer, and give explanations for the distractors for the following passage. Please use the following format:
    Question: [Write the question here.] 
    A. [Write option A here]
    B. [Write option B here]
    C. [Write option C here]
    D. [Write option D here]

    Correct Answer: [Write the correct answer here] 

    Explanation for A: [Write the explanation for A here]
    Explanation for B: [Write the explanation for B here]
    Explanation for C: [Write the explanation for C here]
    Explanation for D: [Write the explanation for D here]
    
"""
    text_input = prompt + summary
    completions = openai.Completion.create(engine="text-davinci-003", prompt=text_input, max_tokens=2048, n=1,stop=None,temperature=0.7)
    message = completions.choices[0].text
    return message

In [7]:
def chatGPTparseMCQ(output):    
    output_dict['question'].append(re.search('(Question:)[\w\d\s]*/?', output).group(0)[10:])
    output = re.sub('(Question:)[\w\d\s]*/?', '', output).strip()

    output_dict['answer_a'].append(re.search('A.+', output).group(0)[3:])
    output = re.sub('A.+', '', output, count=1).strip()
    output_dict['answer_b'].append(re.search('B.+', output).group(0)[3:])
    output = re.sub('B.+', '', output, count=1).strip()
    output_dict['answer_c'].append(re.search('C.+', output).group(0)[3:])
    output = re.sub('C.+', '', output, count=1).strip()
    output_dict['answer_d'].append(re.search('D.+', output).group(0)[3:])
    output = re.sub('D.+', '', output, count=1).strip()

    output_dict['correct_answer'].append(re.search('[Cc]orrect [Aa]nswer: .', output).group(0)[-1])
    output = re.sub('[Cc]orrect [Aa]nswer: .+', '', output, count=1).strip()

    a = re.search('Explanation for A.+', output)
    if a == None:
        output_dict['explanation_a'].append('')
    else:
        output_dict['explanation_a'].append(a.group(0)[18:])

    b = re.search('Explanation for B.+', output)
    if b == None:
        output_dict['explanation_b'].append('')
    else:
        output_dict['explanation_b'].append(b.group(0)[18:])
    
    c = re.search('Explanation for C.+', output)
    if c == None:
        output_dict['explanation_c'].append('')
    else:
        output_dict['explanation_c'].append(c.group(0)[18:])
        
    d = re.search('Explanation for D.+', output)
    if d == None:
        output_dict['explanation_d'].append('')
    else:
        output_dict['explanation_d'].append(d.group(0)[18:])




In [ ]:
output_dict = {'chapter': [], 'section': [], 'subsection_number':[], 'subsection_slug': [], 'question': [], 'answer_a':[], 'answer_b':[], 'answer_c':[], 'answer_d':[], 
                'explanation_a':[], 'explanation_b':[], 'explanation_c':[], 'explanation_d':[], 
                'correct_answer':[]}

for row in subsections.iterrows():
    subsection = row[1]
    output_dict['chapter'].append(subsection['chapter'])
    output_dict['section'].append(subsection['section'])
    output_dict['subsection_number'].append(subsection['subsection'])
    output_dict['subsection_slug'].append(subsection['slug'])
    output = generate_question(subsection['clean_text'])
    print(output)
    chatGPTparseMCQ(output)
    
pd.DataFrame.from_dict(output_dict).to_csv(DATA+'mcq_table.csv')

In [ ]:
pd.DataFrame.from_dict(output_dict)